In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import healpy as hp
from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
import os
path = os.getcwd()

path_mm = os.path.dirname(path)
import sys
sys.path.append(path_mm)
CMB_CL_FILE = '/Users/mregnier/Desktop/PhD Regnier/mypackages/Cls_Planck2018_'

def cl2dl(ell, cl):

    dl=np.zeros(ell.shape[0])
    for i in range(ell.shape[0]):
        dl[i]=(ell[i]*(ell[i]+1)*cl[i])/(2*np.pi)
    return dl
def give_cl_cmb(r=0, Alens=1.):
    power_spectrum = hp.read_cl(CMB_CL_FILE+'lensed_scalar.fits')[:,:4000]
    if Alens != 1.:
        power_spectrum[2] *= Alens
    if r:
        power_spectrum += r * hp.read_cl(CMB_CL_FILE+'unlensed_scalar_and_tensor_r1.fits')[:,:4000]
    return power_spectrum

path = 'data_mm/'

with open(path+f'MM_band220_bandpasscorrectionFalse_Nrec2_Nsub4_Ntod100_correction_conv0.0deg_noiseFalse.pkl', 'rb') as f:
    data = pickle.load(f)
print(data.keys())

In [ ]:
ell = np.arange(2, 4000, 1)
Dls = cl2dl(ell, give_cl_cmb(r=0, Alens=1)[2])

In [ ]:
allnsub = [4, 8]#[5, 10, 15, 20, 30]

plt.figure(figsize=(10, 8))

plt.plot(ell, Dls, label=r'Theoretical CMB | r = 0, $A_{lens}$ = 1')

for ii, i in enumerate(allnsub):
    with open(path+f'MM_band220_bandpasscorrectionFalse_Nrec2_Nsub{i}_Ntod100_correction_conv0.0deg_noiseFalse.pkl', 'rb') as f:
        data = pickle.load(f)
    plt.plot(data['leff'], data['Dl_BB'], '-o', label=f'Nsub = {i}')
plt.xlim(20, 500)
plt.yscale('log')
plt.ylim(1e-5, 1e-1)
plt.legend(frameon=False, fontsize=12)
plt.title(r'$N_{TOD}$' + f' = {100}')

plt.show()

In [ ]:
Dls_binned = np.interp(data['leff'], ell, Dls)

In [ ]:
Dls_bias = np.zeros((len(allnsub), 13))
for ii, i in enumerate(allnsub):
    with open(path+f'MM_band220_bandpasscorrectionFalse_Nrec2_Nsub{i}_Ntod100_correction_conv0.0deg_noiseFalse.pkl', 'rb') as f:
        data = pickle.load(f)
    Dls_bias[ii] = Dls_binned + data['Dl_BB']

In [ ]:
def chi2(theta, d, leff, ell_theo, Dl_theo):
    r = theta
    Alens = 1
    Dl_simu = cl2dl(ell_theo, give_cl_cmb(r=r, Alens=Alens)[2])
    Dl_simu = np.interp(leff, ell_theo, Dl_simu)
    
    return np.sum((Dl_simu - d)**2)

In [ ]:
from scipy.optimize import minimize
r = np.zeros(len(allnsub))
for ii, i in enumerate(allnsub):
    r[ii]=minimize(chi2, x0=np.zeros(1), args=(Dls_bias[ii], data['leff'], ell, Dls), tol=1e-10, method='BFGS').x
    print(r[ii])

In [ ]:
allnsub = [4, 8]#[5, 10, 15, 20, 30]

plt.figure(figsize=(10, 8))

plt.plot(ell, Dls, label=r'Theoretical CMB | r = 0, $A_{lens}$ = 1')

for ii, i in enumerate(allnsub):
    with open(path+f'MM_band220_bandpasscorrectionFalse_Nrec2_Nsub{i}_Ntod100_correction_conv0.0deg_noiseFalse.pkl', 'rb') as f:
        data = pickle.load(f)
    
    plt.plot(data['leff'], data['Dl_BB'], '-o', label=f'Nsub = {i} | '+r'$r_{bias}$ = '+f'{r[ii]:.4f}')

plt.xlim(20, 500)
plt.yscale('log')
plt.ylim(1e-5, 1e-1)
plt.legend(frameon=False, fontsize=12)
plt.title(r'$N_{TOD}$' + f' = {100}')

plt.show()

In [ ]:
ls

# Display Maps

In [ ]:
with open(f'data/MM_band220_bandpasscorrectionTrue_Nrec1_Nsub4_Ntod4_correction_conv2.0deg_noiseTrue_1.pkl', 'rb') as f:
    data = pickle.load(f)
center = data['center']

In [ ]:
plt.figure(figsize=(15, 5))

hp.gnomview(data['input'][0, :, 1], rot=center, reso=15, cmap='jet', min=-8, max=8, 
            sub=(1, 3, 1), notext=True, title='Input')
hp.gnomview(data['output'][0, :, 1], rot=center, reso=15, cmap='jet', min=-8, max=8, 
            sub=(1, 3, 2), notext=True, title='Output')
hp.gnomview(data['output'][0, :, 1]-data['input'][0, :, 1], rot=center, reso=15, cmap='jet', min=-8, max=8,
            sub=(1, 3, 3), notext=True, title='Residual')
plt.show()

In [ ]:
N = 2

m = np.zeros((N, 2, 12*256**2, 3))
for i in range(N):
    with open(f'data/MM_band220_bandpasscorrectionTrue_Nrec2_Nsub4_Ntod4_correction_conv2.0deg_noiseTrue_{i+1}.pkl', 'rb') as f:
        data = pickle.load(f)
    m[i] = data['output'].copy()
index = np.where(data['seenpix'] == True)[0]
coverage = data['coverage']

hp.gnomview(data['input'][0, :, 1]-np.mean(m, axis=0)[0, :, 1], rot=data['center'], reso=15, cmap='jet', min=-10, max=10)

In [ ]:
# FG-Buster packages
import component_model as c
import mixing_matrix as mm
import frequency_acquisition as Acq
import qubic

comp = [c.CMB(), c.Dust(nu0=nueff[0])]
beta = np.array([1.54, 20])
beta_model = np.array([1.54, 20])
allnus = np.linspace(190, 250, 100)

sed = mm.MixingMatrix(*comp).evaluator(allnus)(beta)
sed_model = mm.MixingMatrix(*comp).evaluator(allnus)(beta_model)

In [ ]:
dictfilename = 'dicts/pipeline_demo.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)

d['nf_recon'] = 2
d['nf_sub'] = 2
d['nside'] = 256
npix=12*d['nside']**2
d['RA_center'] = 0
d['DEC_center'] = -57
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
d['effective_duration'] = 3
d['npointings'] = 100
d['filter_nu'] = 220 * 1e9
d['photon_noise'] = True
d['noiseless'] = not True
d['config'] = 'FI'
d['filter_relative_bandwidth'] = 0.25
d['MultiBand'] = True
d['planck'] = True
d['dtheta'] = 15
d['synthbeam_dtype'] = float

qubic_acquisition = Acq.QubicIntegrated(d, Nsub=2, Nrec=2)
cmb = qubic_acquisition.get_PySM_maps({'cmb':42}, np.array([nueff[0]]))
dust = qubic_acquisition.get_PySM_maps({'dust':'d0'}, np.array([nueff[0]]))
components = np.array([cmb[0], dust[0]])

In [ ]:
frequency_maps = sed# @ components[:, :, 0]
#frequency_maps += cmb[0, :, 0]
print(frequency_maps.shape)

In [ ]:
_, _, nueff, _, _, _ = qubic.compute_freq(220, Nfreq=2)

In [ ]:
hp.gnomview(m[0, 0, :, 1], rot=center, reso=15, min=-30, max=30)

In [ ]:
mycut_0_1 = coverage/coverage.max() > 0.1
mycut_0 = coverage/coverage.max() > 0

In [ ]:
index.shape

In [ ]:
ipix = 20000
istk = 1

plt.plot(allnus, sed[:, 0]*components[0, index[ipix], istk]+sed[:, 1]*components[1, index[ipix], istk])

plt.errorbar(nueff, np.mean(m[:, :, index[ipix], istk], axis=0), yerr=np.std(m[:, :, index[ipix], istk], axis=0), fmt='or',
            capsize=3)

#plt.errorbar(nueff, m[0, :, index[ipix], 0], yerr = np.std(m[:, :, mycut_0_1, :], axis=2)[0, :, 0], fmt='ok',
#            capsize=3)

In [ ]:
def mychi2_on_beta(x, nus, d):
    
    print(x)
    comp = [c.Dust(nu0=nus[0], temp=20)]

    sed = mm.MixingMatrix(*comp).evaluator(nus)(x)
    diff = d - sed @ np.array([d[0]])
    #print(d, sed @ np.array([d[0]]))
    return np.sum(diff**2)

In [ ]:
from scipy.optimize import minimize

minimize(mychi2_on_beta, x0=np.array([1.5]), args=(nueff, m[0, :, index[ipix], 1]), tol=1e-10,
        method='L-BFGS-B')